In [2]:
# %load notebook_setup.py
#Use %load notebook_setup.py to load these commands at the top of your
#ipython notebook.  Includes running the std_libraries.py file. 

%run std_libraries.py
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook

what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = otherb
base directory is /home/heineike/github/expression_broad_data
Added /home/heineike/github/expression_broad_data to path: 
['/home/heineike/github/expression_broad_data/scripts', '', '/home/lab/envs/seqanalysis/lib/python35.zip', '/home/lab/envs/seqanalysis/lib/python3.5', '/home/lab/envs/seqanalysis/lib/python3.5/plat-linux', '/home/lab/envs/seqanalysis/lib/python3.5/lib-dynload', '/home/lab/envs/seqanalysis/lib/python3.5/site-packages', '/home/lab/envs/seqanalysis/lib/python3.5/site-packages/argh-0.26.1-py3.5.egg', '/home/lab/envs/seqanalysis/lib/python3.5/site-packages/glob2-0.4.1-py3.5.egg', '/home/lab/envs/seqanalysis/lib/python3.5/site-packages/mmtf_python-1.0.2-py3.5.egg', '/home/lab/envs/seqanalysis/lib/python3.5/site-packages/IPython/extensions', '/home/heineike/.ipython', '/home/heineike/github/expression_broad_data']
Importing expression plots and io_library and setting base_dir and data

In [19]:
# Set gois
    
gois_common_orf = [
 ('RME1','YGR044C'), 
 #('A1','MATA1'),
 ('MATALPHA2','YCR039C'),
 ('GPA1','YHR005C'),
 ('STE4','YOR212W'),
 ('STE11','YLR362W'),
 ('FAR1','YJL157C')
]

gois = [goi_common_orf[1] for goi_common_orf in gois_common_orf]

exp_data = pd.DataFrame(gois_common_orf, columns = ['Scer_common_name', 'Scer_genename'])


,Scer_common_name,Scer_genename
0,RME1,YGR044C
1,MATALPHA2,YCR039C
2,GPA1,YHR005C
3,STE4,YOR212W
4,STE11,YLR362W
5,FAR1,YJL157C


In [42]:

#Add data for just S.Cer


# Induction data

spec_order_post_WGH = ['Spar','Sbay','Smik','Suva', 'Cgla', 'Ncas','Vpol']
spec_order_pre_WGH = ['Klac','Lwal','Lklu', 'Calb', 'Ylip','Sjap','Spom']

spec_order_post_WGH_tsankov = ['Spar','Smik','Sbay', 'Cgla', 'Ncas']
spec_order_pre_WGH_tsankov = ['Klac','Lwal','Lklu','Calb', 'Ylip']
specs_tsankov = spec_order_pre_WGH_tsankov + spec_order_pre_WGH_tsankov

orth_dir = data_processing_dir + 'ortholog_files_regev' + os.sep 
base_spec = 'Scer'
spec_conditions = {}

#Add data for just S.Cer
fname_array_data = data_processing_dir + os.path.normpath('regev_data/raw_exp/' + base_spec + '_raw_exp_norm.csv')  
raw_exp_tsankov_norm = pd.read_csv(fname_array_data, index_col=0, header=None)
spec_data_tsankov = raw_exp_tsankov_norm.to_dict()[1]
raw_exp_vals_list = []

fname_array_data = os.path.normpath(data_processing_dir + 'regev_data/' + base_spec + '_growth_stress_norm.csv')  
spec_data = pd.read_csv(fname_array_data, index_col=0)

#Load Experimental Data
conditions = spec_data.columns
spec_conditions[spec] = conditions
exp_vals_lists = {condition: [] for condition in conditions}

for goi in exp_data['Scer_genename']:   
    
    raw_exp_vals = [spec_data_tsankov[goi]] 
    raw_exp_vals_list.append(raw_exp_vals)
    
    for condition in conditions: 
        exp_vals = [spec_data.loc[goi,condition]]
        exp_vals_lists[condition].append(exp_vals)

exp_data[base_spec + '_tsankov'] = raw_exp_vals_list       
        
for condition in conditions: 
    exp_data[base_spec + '_' + condition] = exp_vals_lists[condition]
    







for spec in spec_order_post_WGH + spec_order_pre_WGH: 

    if spec in specs_tsankov:
        fname_array_data = data_processing_dir + os.path.normpath('regev_data/raw_exp/' + spec + '_raw_exp_norm.csv')  
        raw_exp_tsankov_norm = pd.read_csv(fname_array_data, index_col=0, header=None)
        spec_data_tsankov = raw_exp_tsankov_norm.to_dict()[1]
        raw_exp_vals_list = []
    
    fname_array_data = os.path.normpath(data_processing_dir + 'regev_data/' + spec + '_growth_stress_norm.csv')  
    spec_data = pd.read_csv(fname_array_data, index_col=0)
    
    #load ortholog mapping
    conditions = spec_data.columns
    spec_conditions[spec] = conditions
    exp_vals_lists = {condition: [] for condition in conditions}

    orth_lookup = io_library.read_orth_lookup_table(base_spec, spec, orth_dir)

    #List number of orthologs
    #For each ortholog get expression

    orths_list = []
    Norths_list = []


    for goi in exp_data['Scer_genename']:
        
        if goi in orth_lookup.keys():
            orths = orth_lookup[goi]
        else: 
            print('No ortholog in ' + spec + ' for ' + goi)
            orths = ['NONE']
            
        orths_list.append(orths)
        if orths == ['NONE']:  #there are two ways to get ['NONE'] - explicitly listed in ortholog table, or no key to ortholog table
            Norths = 0
        else: 
            Norths = len(orths)
        Norths_list.append(Norths)
    
        if spec in specs_tsankov:
            raw_exp_vals = []
            for orth in orths: 
                if orth =='NONE':
                    raw_exp_vals.append(np.nan)
                else: 
                    raw_exp_vals.append(spec_data_tsankov[orth]) 
            raw_exp_vals_list.append(raw_exp_vals)
        
        for condition in conditions: 
            exp_vals = []
            for orth in orths: 
                if orth =='NONE':
                    exp_vals.append(np.nan)
                else: 
                    exp_vals.append(spec_data.loc[orth,condition])
            exp_vals_lists[condition].append(exp_vals)

    exp_data[spec + '_Norths'] = Norths_list
    exp_data[spec + '_orths'] = orths_list
    if spec in specs_tsankov: 
        exp_data[spec + '_tsankov'] =  raw_exp_vals_list
    for condition in conditions: 
        exp_data[spec + '_' + condition] = exp_vals_lists[condition]


        
#make table of number of orthologs

#load species data


No ortholog in Ncas for YCR039C


In [39]:
raw_exp_vals_list

[]

In [46]:
exp_data.to_csv('/home/heineike/scratch/20190619_rme1_expression_data.csv')

In [36]:

#load expression data
spec = 'Ncas'
fname_array_data = data_processing_dir + os.path.normpath('regev_data/raw_exp/' + spec + '_raw_exp_norm.csv')  
raw_exp_tsankov_norm = pd.read_csv(fname_array_data, index_col=0, header=None)
spec_data_tsankov = raw_exp_tsankov_norm.to_dict()[1]

spec_data


{'Scas717.4': -2.6326880017292145,
 'Scas718.65': -0.23149791421540836,
 'Scas655.11': 0.06035876913004146,
 'Scas710.30': -0.05904371846155172,
 'Scas608.6': 0.2934683057379875,
 'Scas680.9': -0.029060445332705475,
 'Scas571.7': 1.7145135678089312,
 'Scas646.20': 0.013597976043811863,
 'Scas718.33': 1.0331138854291877,
 'Scas700.39': -0.6514188769347234,
 'Scas696.22': -0.009036387387994137,
 'Scas663.3d': 0.30946803173718485,
 'Scas654.9': -0.19163788061440606,
 'Scas201.1': 1.0862065439877306,
 'Scas565.4': -0.28317762120797585,
 'Scas650.13': -0.19569368528994385,
 'Scas671.19': 0.5339598021307344,
 'Scas653.25': -0.012759314451587535,
 'Scas655.21': -1.0254692922059074,
 'Scas76.1': 0.8119215478645625,
 'Scas548.5': 0.09701387209892652,
 'Scas470.1': 0.12145985576563507,
 'Scas720.47': 0.3967730385570336,
 'Scas691.34': 1.5513516013166329,
 'Scas658.23': 1.85167596785394,
 'Scas679.17': 0.1167471593644662,
 'Scas665.19': -0.22312062292825693,
 'Scas701.44': -1.5059505883607427,
 '

In [ ]:
# Expression Data

In [ ]:
 #Must use ortholog mapping for regev data. 
    orth_dir = data_processing_dir + 'ortholog_files_regev' + os.sep 

    columns_to_keep=[]
    for level in ['low', 'high']:
        for column_base in ['genename', sort_column]:
            columns_to_keep.append(column_base + '_' + level)

    ohnologs_goi_array = ohnologs_goi.loc[:,columns_to_keep]
    ohnologs_goi_array.rename(columns = {'genename_'+level : self_spec+'_genename_'+level for level in ['low','high']}, inplace=True)

    spec_conditions = {}

    #adding data just for self_spec

    #load data
    fname_array_data = os.path.normpath(data_processing_dir + 'regev_data/' + self_spec + '_growth_stress_norm.csv')  
    spec_data = pd.read_csv(fname_array_data, index_col=0)
    conditions = spec_data.columns
    spec_conditions[self_spec] = conditions

    for level in ['low','high']:
        condition_data_dict = {condition: [] for condition in conditions}
        for gene in ohnologs_goi_array[self_spec + '_genename_' + level ]:
            for condition in conditions:
                cond_data = []
                try: 
                    cond_data.append(spec_data.loc[gene,condition])
                except KeyError:
                    cond_data.append(np.nan)
                    print('Mismatch between goi index and expression data index ' + self_spec + ' : ' + gene + ' ' + condition)
                condition_data_dict[condition].append(cond_data)

        for condition in conditions: 
            ohnologs_goi_array[self_spec + '_' + condition + '_' + level] = condition_data_dict[condition]



    #post WGH:


    for spec in spec_order_post_WGH: 

        #load ortholog mapping
        orth_lookup = read_orth_lookup_table(self_spec, spec, orth_dir)

        #load data
        fname_array_data = os.path.normpath(data_processing_dir + 'regev_data/' + spec + '_growth_stress_norm.csv')  
        spec_data = pd.read_csv(fname_array_data, index_col=0)
        conditions = spec_data.columns
        spec_conditions[spec] = conditions

        for level in ['low','high']: 
            N_orth_list = []
            orth_name_list = []
            condition_data_dict = {condition: [] for condition in conditions}
            for gene in ohnologs_goi_array[self_spec + '_genename_' + level ]:
                try: 
                    if orth_lookup[gene][0]=='NONE':
                        N_orth_list.append(0)
                        orth_name_list.append('NONE_in_orthogroups')
                        for condition in conditions: 
                            condition_data_dict[condition].append(np.nan)
                        print(gene + ' has NONE listed in ortholog file for ' + spec)
                    else: 
                        orth_names = orth_lookup[gene]
                        N_orth_list.append(len(orth_names))
                        orth_name_list.append(orth_names)
                        for condition in conditions:
                            cond_data = []
                            for spec_gene in orth_names: 
                                try: 
                                    cond_data.append(spec_data.loc[spec_gene,condition])
                                except KeyError:
                                    cond_data.append(np.nan)
                                    print('Mismatch between ortholog file and expression data index ' + spec + ' : ' + spec_gene + ' ' + condition)
                            condition_data_dict[condition].append(cond_data)
                except KeyError:
                    N_orth_list.append(0)
                    orth_name_list.append('NONE_not_in_orthogroups')
                    for condition in conditions: 
                        condition_data_dict[condition].append(np.nan)
                    print(gene + ' not present in orthogroup file for ' + spec)

            ohnologs_goi_array[spec + '_N_' + level] = N_orth_list
            ohnologs_goi_array[spec + '_genename_' + level] = orth_name_list
            for condition in conditions: 
                ohnologs_goi_array[spec + '_' + condition + '_' + level] = condition_data_dict[condition]



    #pre WGH:


    for spec in spec_order_pre_WGH: 

        #load ortholog mapping
        orth_lookup = read_orth_lookup_table(self_spec, spec, orth_dir)

        #load data
        fname_array_data = os.path.normpath(data_processing_dir + 'regev_data/' + spec + '_growth_stress_norm.csv')  
        spec_data = pd.read_csv(fname_array_data, index_col=0)
        conditions = spec_data.columns
        spec_conditions[spec] = conditions

        N_orth_list = []
        orth_name_list = []
        condition_data_dict = {condition: [] for condition in conditions}

        for row in ohnologs_goi_array.loc[:,[self_spec + '_genename_low',self_spec + '_genename_high']].iterrows():

            gene_low = row[1][self_spec+'_genename_low']
            gene_high = row[1][self_spec+'_genename_high']

            try:         
                if orth_lookup[gene_low] == orth_lookup[gene_high]: 
                    gene_to_test = gene_low
                    if orth_lookup[gene_to_test][0]=='NONE':
                        N_orth_list.append(0)
                        orth_name_list.append('NONE_in_orthogroups')
                        for condition in conditions: 
                            condition_data_dict[condition].append(np.nan)
                        print(gene_to_test + ' has NONE listed in ortholog file for ' + spec)
                    else: 
                        orth_names = orth_lookup[gene_to_test]
                        N_orth_list.append(len(orth_names))
                        orth_name_list.append(orth_names)
                        for condition in conditions:
                            cond_data = []
                            for spec_gene in orth_names:
                                try: 
                                    cond_data.append(spec_data.loc[spec_gene,condition])
                                except KeyError:
                                    cond_data.append(np.nan)
                                    print('Mismatch between ortholog file and expression data index ' + spec + ' : ' + spec_gene + ' ' + condition)
                            condition_data_dict[condition].append(cond_data)

                else: 
                    raise LookupError('high and low paralogs do not have same ortholog in ' + spec + ': ' + gene_low + ' and ' + gene_high )
            except KeyError: 
                print('either ' + gene_low + ' or ' + gene_high + 'is not in index for ortholog table in ' + spec)    


                gene_to_test = 'NONE'
                if gene_low in orth_lookup.keys():
                    print(gene_low + ' is in index for lookup table')
                    gene_to_test = gene_low
                elif gene_high in orth_lookup.keys():
                    print(gene_high + ' is in index for lookup table')
                    gene_to_test = gene_high

                if gene_to_test != 'NONE':
                    print(gene_to_test + ' did have an ortholog in ' + spec)
                    if orth_lookup[gene_to_test][0]=='NONE':
                        N_orth_list.append(0)
                        orth_name_list.append('NONE_in_orthogroups')
                        for condition in conditions: 
                            condition_data_dict[condition].append(np.nan)
                        print(gene_to_test + ' has NONE listed in ortholog file for ' + spec)
                    else: 
                        orth_names = orth_lookup[gene_to_test]
                        N_orth_list.append(len(orth_names))
                        orth_name_list.append(orth_names)
                        for condition in conditions:
                            cond_data = []
                            for spec_gene in orth_names:
                                try: 
                                    cond_data.append(spec_data.loc[spec_gene,condition])
                                except KeyError:
                                    cond_data.append(np.nan)
                                    print('Mismatch between ortholog file and expression data index ' + spec + ' : ' + spec_gene + ' ' + condition)
                            condition_data_dict[condition].append(cond_data)
                else:
                    raise RuntimeError('Neither gene is index for ortholog table, but it was not caught in the first section')


        ohnologs_goi_array[spec + '_N'] = N_orth_list
        ohnologs_goi_array[spec + '_genename'] = orth_name_list
        for condition in conditions: 
            ohnologs_goi_array[spec + '_' + condition] = condition_data_dict[condition]
